Click [here]() to access the associated Medium article.

# Setup

In [4]:
import math
from tensor import Tensor
from time import time_function
from collections import List

# Bloom Filter

In [5]:
struct BloomFilter:
    var items_count: Int
    var fp_prob: Float32
    var size: Int
    var hash_count: Int
    var bit_array: Tensor[DType.bool]

    fn __init__(inout self, items_count: Int, fp_prob: Float32) -> NoneType:
        """Initialize Bloom filter.

        Args:
            items_count: Number of items expected to be stored in bloom filter.
            fp_prob: False Positive probability in decimal.
        """
        self.items_count = items_count
        self.fp_prob = fp_prob
        self.size = int(-items_count * math.log(fp_prob) / math.log(2.0) ** 2)
        self.hash_count = int((self.size / items_count) * math.log(2.0))
        self.bit_array = Tensor[DType.bool](self.size)

    fn insert(inout self, item: String) -> NoneType:
        """Insert an item to the filter."""
        for i in range(self.hash_count):
            var digest = int((hash(item) + i) % self.size)
            self.bit_array[digest] = True

    fn has(self, item: String) -> Bool:
        """Check for existence of an item in filter."""
        for i in range(self.hash_count):
            var digest = int((hash(item) + i) % self.size)
            # If any of the bits in the bit array is False, then the item
            # is not present in the filter. Otherwise, there is a probability
            # that it exists.
            if self.bit_array[digest] == False:
                return False
        return True

    fn get_load_factor(self) -> Float32:
        """Return the load factor of the filter."""
        return self.items_count / self.size


# Benchmark

In [6]:
fn benchmark1() capturing -> NoneType:
    """Benchmark 1: Insert 1,000,000 items."""
    var bloom = BloomFilter(items_count=1_000_000, fp_prob=0.01)

    print("Size of bit array:", bloom.size)
    print("False positive Probability:", bloom.fp_prob)
    print("Number of hash functions:", bloom.hash_count)
    print("Load factor:", bloom.get_load_factor())

    for i in range(bloom.items_count):
        bloom.insert(String(i))


fn benchmark2() capturing -> NoneType:
    """Benchmark 2: Test the Bloom Filter with sample words."""
    var bloom = BloomFilter(items_count=7, fp_prob=0.2)

    print("Size of bit array:", bloom.size)
    print("False positive Probability:", bloom.fp_prob)
    print("Number of hash functions:", bloom.hash_count)
    print("Load factor:", bloom.get_load_factor(), "\n")

    # Insert word_present to Bloom Filter
    var word_present = List(
        "abound",
        "abounds",
        "abundance",
        "abundant",
        "accessible",
        "bloom",
        "blossom",
    )
    var word_absent = List("bluff", "cheater", "hate")
    for word in word_present:
        bloom.insert(word[])

    # Test presence of words
    var definitely_not_present = List[String]()
    var probably_present = List[String]()
    var false_positive = List[String]()
    var test_words = word_present + word_absent

    for word in test_words:
        if bloom.has(word[]):
            if word[] in word_absent:
                false_positive.append(word[])
            else:
                probably_present.append(word[])
        else:
            definitely_not_present.append(word[])

    print("Definitely not present:\t", end=" ")
    for word in definitely_not_present:
        print(word[], end=", ")

    print("\nProbably present:\t", end=" ")
    for word in probably_present:
        print(word[], end=", ")

    print("\nFalse positive:\t\t", end=" ")
    for word in false_positive:
        print(word[], end=", ")
    print()


In [7]:
print("-----------------------------------")
print("Benchmark 1: Insert 1,000,000 items")
print("-----------------------------------")
var time_taken = time_function[benchmark1]() / 10e9
print("Time taken:", time_taken, "seconds")

print("\n")

print("----------------------------------------------------")
print("Benchmark 2: Test the Bloom Filter with sample words")
print("----------------------------------------------------")
benchmark2()


-----------------------------------
Benchmark 1: Insert 1,000,000 items
-----------------------------------
Size of bit array: 9585058
False positive Probability: 0.0099999997764825821
Number of hash functions: 6
Load factor: 0.10432904958724976
Time taken: 0.14369109999999999 seconds


----------------------------------------------------
Benchmark 2: Test the Bloom Filter with sample words
----------------------------------------------------
Size of bit array: 23
False positive Probability: 0.20000000298023224
Number of hash functions: 2
Load factor: 0.30434781312942505 

Definitely not present:	 bluff, cheater, 
Probably present:	 abound, abounds, abundance, abundant, accessible, bloom, blossom, 
False positive:		 hate, 
